In [1]:
import React from 'react';
import Highcharts from 'highcharts/highstock';
import HighchartsReact from 'highcharts-react-official';

const StockChart = ({ data }) => {
  const options = {
    title: {
      text: 'Stock Price'
    },
    xAxis: {
      type: 'datetime'
    },
    yAxis: {
      title: {
        text: 'Price'
      }
    },
    series: [{
      name: 'Stock Price',
      data: data,
      tooltip: {
        valueDecimals: 2
      }
    }]
  };

  return (
    <HighchartsReact
      highcharts={Highcharts}
      constructorType={'stockChart'}
      options={options}
    />
  );
};

export default StockChart;

SyntaxError: invalid syntax (2490503976.py, line 1)

In [ ]:
 if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatHeader}>
          <div className={styles.chatHeaderTitle}>
            <div className={styles.iconWrapper}>
              <FluentIcons.Sparkle24Filled />
            </div>
            <Text className={styles.chatHeaderText}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text>
        </div>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <React.Fragment key={index}>
              <div
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                )}
              </div>
              {message.role === "user" && index === 0 && isFirstQuery && (
                <div className={styles.chartContainer}>
                  <StockChart data={stockData} />
                </div>
              )}
              {message.subgraphResults && (
                <div className={styles.subgraphMessage}>
                  <SubgraphResults results={message.subgraphResults} />
                </div>
              )}
            </React.Fragment>
          ))}
        </div>

In [ ]:
from typing import Dict, List
from langgraph.graph import StateGraph, END

# Define your functions
def vector_db_search(query: str) -> str:
    # Implement your vector database search here
    return f"Vector DB result for: {query}"

def query_expansion(query: str) -> List[str]:
    # Implement your query expansion logic here
    return [f"{query} expanded 1", f"{query} expanded 2"]

def api_call(query: str) -> str:
    # Implement your API call here
    return f"API result for: {query}"

# Define the routing logic
def route_query(state):
    query = state['query']
    if len(query.split()) > 3:  # Simple routing logic, adjust as needed
        return "query_expansion"
    else:
        return "vector_db_search"

# Define the query expansion and API call logic
def expand_and_call(state):
    expanded_queries = query_expansion(state['query'])
    results = []
    for eq in expanded_queries:
        results.append(api_call(eq))
    return {"expanded_results": results}

# Define the aggregation logic
def aggregate_results(state):
    if "expanded_results" in state:
        # Aggregate expanded results
        combined = " ".join(state["expanded_results"])
        return {"response": f"Aggregated results: {combined}"}
    else:
        # Return vector DB result
        return {"response": state["result"]}

# Create the graph
workflow = StateGraph(name="QueryProcessor")

# Add nodes
workflow.add_node("route", route_query)
workflow.add_node("vector_db_search", lambda state: {"result": vector_db_search(state['query'])})
workflow.add_node("expand_and_call", expand_and_call)
workflow.add_node("aggregate", aggregate_results)

# Add edges
workflow.add_edge("route", "vector_db_search")
workflow.add_edge("route", "expand_and_call")
workflow.add_edge("vector_db_search", "aggregate")
workflow.add_edge("expand_and_call", "aggregate")
workflow.add_edge("aggregate", END)

# Set the entrypoint
workflow.set_entry_point("route")

# Compile the graph
app = workflow.compile()

# Example usage
query = "What is the capital of France?"
result = app.invoke({"query": query})
print(result['response'])

In [ ]:
from PyPDF2 import PdfReader
from pptx import Presentation
import os

def read_pdf(pdf_path):
    """
    Read text content from a PDF file.
    
    Args:
        pdf_path (str): Path to the PDF file
        
    Returns:
        dict: Dictionary containing page numbers and their text content
    """
    result = {
        'total_pages': 0,
        'content': {},
        'metadata': {}
    }
    
    try:
        # Open the PDF
        reader = PdfReader(pdf_path)
        
        # Get basic information
        result['total_pages'] = len(reader.pages)
        result['metadata'] = reader.metadata
        
        # Extract text from each page
        for page_num, page in enumerate(reader.pages, 1):
            text = page.extract_text()
            result['content'][page_num] = text.strip()
            
        return result
        
    except Exception as e:
        print(f"Error reading PDF {pdf_path}: {str(e)}")
        return None

def read_pptx(pptx_path):
    """
    Read content from a PowerPoint presentation.
    
    Args:
        pptx_path (str): Path to the PowerPoint file
        
    Returns:
        dict: Dictionary containing slides information and their content
    """
    result = {
        'total_slides': 0,
        'slides': {},
        'notes': {},
        'shapes_count': {},
        'metadata': {
            'author': None,
            'modified': None,
            'created': None,
            'title': None
        }
    }
    
    try:
        # Load presentation
        prs = Presentation(pptx_path)
        
        # Get basic information
        result['total_slides'] = len(prs.slides)
        
        # Get core properties
        if prs.core_properties:
            result['metadata']['author'] = prs.core_properties.author
            result['metadata']['modified'] = prs.core_properties.modified
            result['metadata']['created'] = prs.core_properties.created
            result['metadata']['title'] = prs.core_properties.title
        
        # Process each slide
        for slide_num, slide in enumerate(prs.slides, 1):
            slide_content = []
            notes_content = []
            
            # Get shape count
            result['shapes_count'][slide_num] = len(slide.shapes)
            
            # Extract text from shapes
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    if shape.text.strip():
                        slide_content.append(shape.text.strip())
            
            # Get notes if they exist
            if slide.notes_slide and slide.notes_slide.notes_text_frame:
                notes_content.append(slide.notes_slide.notes_text_frame.text.strip())
            
            # Store the content
            result['slides'][slide_num] = slide_content
            if notes_content:
                result['notes'][slide_num] = notes_content
        
        return result
        
    except Exception as e:
        print(f"Error reading PowerPoint {pptx_path}: {str(e)}")
        return None

def print_document_summary(doc_path):
    """
    Print a summary of the document content based on file type.
    
    Args:
        doc_path (str): Path to the document file
    """
    _, ext = os.path.splitext(doc_path.lower())
    
    if ext == '.pdf':
        result = read_pdf(doc_path)
        if result:
            print("\nPDF Summary:")
            print(f"Total Pages: {result['total_pages']}")
            print("\nMetadata:")
            for key, value in result['metadata'].items():
                if value:
                    print(f"{key}: {value}")
            print("\nContent Preview (first page):")
            if result['content'].get(1):
                preview = result['content'][1][:200] + "..." if len(result['content'][1]) > 200 else result['content'][1]
                print(preview)
                
    elif ext == '.pptx':
        result = read_pptx(doc_path)
        if result:
            print("\nPowerPoint Summary:")
            print(f"Total Slides: {result['total_slides']}")
            print("\nMetadata:")
            for key, value in result['metadata'].items():
                if value:
                    print(f"{key}: {value}")
            print("\nContent Preview (first slide):")
            if result['slides'].get(1):
                print("\n".join(result['slides'][1][:3]))  # Show first 3 text elements
                if len(result['slides'][1]) > 3:
                    print("...")
            if result['notes'].get(1):
                print("\nNotes from first slide:")
                print(result['notes'][1][0][:200] + "..." if len(result['notes'][1][0]) > 200 else result['notes'][1][0])
    
    else:
        print(f"Unsupported file type: {ext}")

# Example usage
if __name__ == "__main__":
    # Reading a PDF
    pdf_path = "example.pdf"
    pdf_content = read_pdf(pdf_path)
    
    # Reading a PowerPoint
    pptx_path = "example.pptx"
    pptx_content = read_pptx(pptx_path)
    
    # Print summaries
    print_document_summary(pdf_path)
    print_document_summary(pptx_path)
    
    # Example of accessing specific content
    if pdf_content:
        # Get text from page 2
        page_2_text = pdf_content['content'].get(2, "Page not found")
        
    if pptx_content:
        # Get text from slide 3
        slide_3_content = pptx_content['slides'].get(3, [])
        slide_3_notes = pptx_content['notes'].get(3, [])